In [1]:
import os
import pickle
import sys
import warnings

import pandas as pd
import plotly.express as px
sys.path.append("../../")
from assets.settings import N_COUNTS_FOR_PRED
from Modules.regression_funcs.rename_columns import rename_columns

warnings.filterwarnings('ignore')


predict_data = pd.Timestamp('2021-11-25')

pred_df = pd.read_parquet('../../Data/prepared/predict/main_predict.gzip')

models = pickle.load(open('../../Data/models/final_models/main_model.pkl', 'rb'))

result = []

for i in range(1, N_COUNTS_FOR_PRED + 1):
    
    prediction = pred_df[['КлиентКод', 'Артикул']]
    prediction['Период'] = predict_data + pd.Timedelta(days=7*(i - 1))
    
    model = models[f'{i}_model']
    
    prediction['prediction'] = model.predict(rename_columns(pred_df)[model.feature_name()])
    
    result.append(prediction)
    
result = pd.concat(result, ignore_index=True)
result

valid = pd.read_parquet('../../Data/prepared/predict/valid_data.gzip')


result = result.merge(valid[['КлиентКод', 'Артикул', 'Период', 'BaseLine_1_week']], how='left', on=['КлиентКод', 'Артикул', 'Период']).rename(columns={"BaseLine_1_week": 'y_true'})

plot_df = result.groupby('Период')[['prediction', 'y_true']].sum().reset_index()


fig = px.scatter()

fig.add_scatter(
    x=plot_df['Период'],
    y=plot_df['prediction'],
    name='Prediction'
)

fig.add_scatter(
    x=plot_df['Период'],
    y=plot_df['y_true'],
    name='Y true',
    fill='tonexty'
)